# About

Throughtout this project, you will be working with the Magnetic Resonance Imaging (MRI) data from The Cancer Genome Atlas (TCGA). The data consists of pre-operative multimodal MR imaging sequences (i.e., T1, T1-Gd, T2, T2-FLAIR) along with their whole-tumor segmentation labels composed of edema, enhancing tumor, and non-enhancing tumor. We have combined these labels into a single whole tumor for this project. Each patient is associated with 144 pre-operative MRI scans, which were treated as independent data points for 2D segmentation. These MRI scans were cropped to 144 × 144 pixels and further pre-processed the data by pixel-intensity normalization. In this notebook, we will access the validation dataset and test datasets.

# Learning Objectives

> In this file, you will learn the following:
>    
><ol>
>  <li>The problem of Brain Tumor (Glioma) Segmentation</li>
>  <li>Glioma and Subtypes</li>
>  <li>Exploring Patients' MRI and Clinical Data</li>
>  <li>Visualizing Examples of MRI Scans and Annotated Tumor Segmentation</li>
></ol>

## Imports

In [39]:
import pandas as pd
from dython.nominal import associations

## 1. The problem of Brain Tumor (Glioma) Segmentation

## 2. Glioma and Subtypes

In [49]:
#[TODO]
#Now, we will take a look at the different types of tumor that the dataset contains.
#For this, we will only look at the columns: 'Tissue source site', 'Study', 'Histology', 'Grade'
df = pd.read_csv('./data/patient_info.csv')
df = df[['Tissue source site', 'Study', 'Histology', 'Grade']]
df

,Tissue source site,Study,Histology,Grade
0,Case Western - St Joes,Brain Lower Grade Glioma,astrocytoma,G3
1,Thomas Jefferson University,Brain Lower Grade Glioma,astrocytoma,G3
2,MD Anderson Cancer Center,Glioblastoma multiforme,glioblastoma,G4
3,Duke,Glioblastoma multiforme,glioblastoma,G4
4,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4
...,...,...,...,...
70,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4
71,MD Anderson Cancer Center,Glioblastoma multiforme,glioblastoma,G4
72,Case Western - St Joes,Brain Lower Grade Glioma,oligodendroglioma,G2
73,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4


In [50]:
#[TODO]
#What are the subtypes of Lower Grade Glioma? Subtypes of Glioblastoma multiforme?
#How are these different subtypes graded?

array(['glioblastoma'], dtype=object)

In [51]:
df.loc[df['Study'] == 'Brain Lower Grade Glioma']['Histology'].unique()
df.loc[df['Study'] == 'Glioblastoma multiforme']['Histology'].unique()

array(['glioblastoma'], dtype=object)

In [52]:
#[TODO]
#Are there any correlations between Tissue source site, Study, Histology and Grade?

## 3. Exploring Patients' MRI and Clinical Data

In [15]:
df = pd.read_csv('./data/patient_info.csv')
df

,Case,Tissue source site,Study,Histology,Grade,Age (years at diagnosis),Gender,Survival (months),Vital status (1=dead)
0,TCGA-HT-A5RC,Case Western - St Joes,Brain Lower Grade Glioma,astrocytoma,G3,70.0,female,5.322494,1.0
1,TCGA-CS-4941,Thomas Jefferson University,Brain Lower Grade Glioma,astrocytoma,G3,67.0,male,7.688047,1.0
2,TCGA-02-0060,MD Anderson Cancer Center,Glioblastoma multiforme,glioblastoma,G4,66.0,female,6.012447,1.0
3,TCGA-12-1093,Duke,Glioblastoma multiforme,glioblastoma,G4,66.0,female,15.967481,1.0
4,TCGA-06-0241,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4,65.0,female,14.948980,0.0
...,...,...,...,...,...,...,...,...,...
70,TCGA-06-0175,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4,69.0,male,4.041153,1.0
71,TCGA-02-0003,MD Anderson Cancer Center,Glioblastoma multiforme,glioblastoma,G4,50.0,male,4.731106,1.0
72,TCGA-HT-8113,Case Western - St Joes,Brain Lower Grade Glioma,oligodendroglioma,G2,49.0,female,29.569410,0.0
73,TCGA-06-0648,Henry Ford Hospital,Glioblastoma multiforme,glioblastoma,G4,77.0,male,9.790760,1.0


In [18]:
#[TODO]
#How many patients have Lower Grade Glioma and how many have Glioblastoma multiforme?

In [19]:
#[TODO]
#What are the different histologies in both types of tumor?

In [20]:
#[TODO]
#What is the mean survival (months) for different types of tumor?

In [21]:
#[TODO]
#Which tissue source site has the most number of patients?

In [22]:
#[TODO]
#What is the median age at diagnosis?

In [23]:
#[TODO]
#Does the gender of the patient have an effect on survival?

## 4. Visualising Patient Cases

In [24]:
#[TODO]
#Pick a patient who identifies as female and has a low survival (months) with high grade tumor.
#What does the MRI scan look like?
#What does the tumor look like?

In [25]:
#[TODO]
#Out of all patients with low grade glioma, which patient has the largest tumor?
#What are the patient characteristics?
#What do the MRI Scans and the tumor look like?

In [27]:
#[TODO]
#Compute the size of the tumor for all patients with Glioblastoma multiforme.
#Is the tumor size correlated with any patient characteristics?